In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
df= pd.read_csv('../data/processed/nlp/cleaned_data.csv')

In [3]:
df.head()

,company_name,title,description,location,tokens
0,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,"Princeton, NJ","['job', 'descriptiona', 'lead', 'real', 'estat..."
1,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...","Fort Collins, CO","['aspen', 'therapy', 'wellness', 'commit', 'se..."
2,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,"Cincinnati, OH","['national', 'exemplar', 'accept', 'applicatio..."
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,"New Hyde Park, NY","['senior', 'associate', 'attorney', 'elder', '..."
4,NaN,Service Technician,Looking for HVAC service tech with experience ...,"Burlington, IA","['looking', 'hvac', 'service', 'tech', 'experi..."


TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
df["text"] = df["tokens"].apply(lambda x: " ".join(x))

vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=5,
    max_df=0.8,
    ngram_range=(1,2)
)

X_tfidf = vectorizer.fit_transform(df["text"])

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

KeyboardInterrupt: 